# Bead with friction

As opposed to the last notebook, this bead is subjected to friction.

In [1]:
import sympy
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, Video, HTML, update_display
import time
from typing import Literal

import sys
sys.path += ["../src", "./src"]

import solvers
import geometry
import animation

In [2]:
sympy.var("x y", real=True)

curve = geometry.CurveConstraint(sympy.sinh(x)**2 + sympy.sinh(y)**2 - 1, x, y, inside="negative", xlim=(-1.5,1.5))

In [3]:
gravity = lambda q, v: np.array([0, -3])

ball = geometry.Ball(curve, mass=1, gravity=gravity, curve_friction=0.3, normal_cor=0.3)

In [4]:
t_end = 5
steps = int(80*t_end)

status = widgets.HTML("-")
display(status)

anim = animation.BallAnimation(curve, [ball], steps)


def callback(step, new, error):

    qnew = new[0:2]
    vnew = new[3:5]
    
    status.value = F'<b>step {step} of {steps}</b> <progress value="{step}" max="{steps}"> </progress>'
    anim.update(qnew, vnew, 0, step-1, "dae")
    

q0=(1, 0.5)
# ensure consistent starting values
q0, v0 = ball._curve.start_projection(q0, v_up = -1.3)

normal = - ball._curve.normal_vec(q0)
a0 = np.pad(ball._curve.curvature(q0) * np.dot(v0, v0) * normal, (0, 1), "constant")

solvers.newmark(q0, v0, a0, ball.mass_mat_dae, ball.damping_mat, ball.force, steps, t_end, callback)

HTML(value='-')

(array([-3.34144287e-01, -8.38509782e-01,  9.85444246e+02, -1.42340653e+00,
         2.57414278e-01,  1.78993518e+07, -7.08283906e+02, -2.54681571e+03,
         3.12219188e+11]),
 400)

In [5]:
# play
HTML(anim.render(1/60))

/home/norberth/venveria/venvs/sc/lib/python3.13/site-packages/matplotlib/cbook.py:1719: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/home/norberth/venveria/venvs/sc/lib/python3.13/site-packages/matplotlib/cbook.py:1355: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


HTML(value='-')